<a href="https://colab.research.google.com/github/NazarovMM/RAG-system-for-dissertation/blob/main/Evaluating_Results_Through_Promt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
import pandas as pd

In [ ]:
# Prompt the user for their OpenAI API key
api_key = input("Please enter your OpenAI API key: ")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Optionally, check that the environment variable was set correctly
print("OPENAI_API_KEY has been set!")

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/NazarovMM/RAG-system-for-dissertation/refs/heads/main/results0-50.csv') # импорт результатов

In [ ]:
df.head(5)

,Prompt,Answer,model_response
0,If my future wife has the same first name as t...,Jane Ballou,"The 15th First Lady of the United States, Edit..."
1,Imagine there is a building called Bronte towe...,37th,The Dewey Decimal Classification (DDC) for a C...
2,How many years earlier would Punxsutawney Phil...,87,The U.S. Capitol's current recognizable state ...
3,"As of August 1, 2024, which country were holde...",France,"As of August 1, 2024, the last time a London-b..."
4,What is the name of the vocalist from the firs...,Jens Kidman,The first band to enter the top 200 under Nucl...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Prompt          50 non-null     object
 1   Answer          50 non-null     object
 2   model_response  50 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [ ]:
prompt="""===Task===

I need your help in evaluating an answer provided by an LLM against a ground truth answer. Your task is to determine if the ground truth answer is present in the LLM’s response. Please analyze the provided data and make a decision.
===Instructions===
1. Carefully compare the "Predicted Answer" with the "Ground Truth Answer".
2. Consider the substance of the answers – look for equivalent information or correct answers. Do not focus on exact wording unless the exact wording is crucial to the meaning.
3. Your final decision should be based on whether the meaning and the vital facts of the "Ground Truth Answer" are present in the "Predicted Answer:"

===Input Data===
- Question: «{question}»
- Predicted Answer: «{answer}»
- Ground Truth Answer: «{ground_truth}»

===Output Format===
Provide your final evaluation in the following format:
"Explanation:" (How you made the decision?)
"Decision:" ("TRUE" or "FALSE" )

Please proceed with the evaluation. """
# эталонный промт из статьи google frames

In [ ]:
question=df['Prompt']
answer=df['model_response']
ground_truth=df['Answer']
test_res=[]

In [ ]:
for i in range(30):
  test_res.append(prompt.format(question=question[i],answer=answer[i],ground_truth=ground_truth[i]))
print(test_res[0])

===Task=== 

I need your help in evaluating an answer provided by an LLM against a ground truth answer. Your task is to determine if the ground truth answer is present in the LLM’s response. Please analyze the provided data and make a decision. 
===Instructions=== 
1. Carefully compare the "Predicted Answer" with the "Ground Truth Answer". 
2. Consider the substance of the answers – look for equivalent information or correct answers. Do not focus on exact wording unless the exact wording is crucial to the meaning. 
3. Your final decision should be based on whether the meaning and the vital facts of the "Ground Truth Answer" are present in the "Predicted Answer:" 

===Input Data=== 
- Question: «If my future wife has the same first name as the 15th first lady of the United States' mother and her surname is the same as the second assassinated president's mother's maiden name, what is my future wife's name? » 
- Predicted Answer: «The 15th First Lady of the United States, Edith Wilson, ha

In [ ]:
result=[]

In [ ]:
for i in range(30):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": test_res[i]
            }
        ]
    )
    result.append(completion.choices[0].message.content)

In [ ]:
result[0]

'Explanation: The predicted answer identifies the first lady (Edith Wilson) and her mother’s name (Sarah), as well as the second assassinated president (James A. Garfield) and his mother’s maiden name (Ballou). It concludes that the future wife\'s name would be "Sarah Ballou." However, the ground truth answer lists the future wife\'s name as "Jane Ballou." Since "Sarah" is not equivalent to "Jane," the predicted answer does not contain the correct name as specified in the ground truth answer, thus failing to present the vital facts correctly.\n\nDecision: FALSE'

In [ ]:
def extract_decision(text): # сохранять только true и false из результатов
    text=text.replace("**","")
    decision_start = text.find("Decision: ") + len("Decision: ")
    decision = text[decision_start:].strip('"')
    return decision

In [ ]:
decision=[]

In [ ]:
for i in range(len(result)):
  decision.append(extract_decision(result[i]))

In [ ]:
res_df=pd.DataFrame(decision,columns=['decision'])
res_df

,decision
0,FALSE
1,FALSE
2,FALSE
3,TRUE
4,TRUE
5,TRUE
6,FALSE
7,FALSE
8,FALSE
9,TRUE


In [ ]:
res_df.to_csv('promtCheck.csv',index=False)